In [ ]:
!pip install wikidata-dl
!pip install pandas SPARQLWrapper requests

In [1]:
import os
import json
import ast

import math
import time
import re
import sys
import logging
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass
from SPARQLWrapper import SPARQLWrapper, JSON
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import pickle
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
    roc_auc_score
)
from sklearn.linear_model import LogisticRegression
import random

from typing import List, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True

2026-02-09 00:14:42.919943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770596085.721187 3845629 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770596086.357884 3845629 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-09 00:14:50.939127: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df_preprocessed = pd.read_csv('data/df_preprocessed.csv')
df_preprocessed

,article_id,theme,concept,term,location,date_x,date_span,phase,phrase_corrected,phrase,...,sentences,sentiment,publication_date,date_y,contexte_enrichi,metadata,context_enrichi_meta,date,event_description,label
0,0,sociopolitique,insécurité,insécurité,"{'text': 'Ouagadougou', 'label': 'departement'}",2009-01-15,"(0, 0)",precrise,Insécurité alimentaire en milieu urbain : La v...,Insécurité alimentaire en milieu urbain : La v...,...,['Insécurité alimentaire en milieu urbain : La...,True,2009-01-15,2009-01-15,Insécurité alimentaire en milieu urbain : La v...,2009-01-15,Publié le 2009-01-15 au Burkina : Insécurité a...,"{'text': '2009-01-15', 'start': 0, 'end': 0}",insécurité - Ouagadougou - 2009-01-15,1
1,0,environment,sécheresse,sécheresse,"{'text': 'Sahel', 'label': 'region'}",2009-01-14,"(261, 279)",precrise,Le Comité permanent inter-Etats de lutte contr...,Le Comité permanent inter-Etats de lutte contr...,...,['Insécurité alimentaire en milieu urbain : La...,False,2009-01-15,2009-01-14,Insécurité alimentaire en milieu urbain : La v...,2009-01-15,Publié le 2009-01-15 au Burkina : Insécurité a...,"{'text': 'le 14 janvier 2009', 'start': 261, '...",sécheresse - Sahel - le 14 janvier 2009,0
2,0,sociopolitique,vulnérabilité,vulnérabilité,"{'text': 'Sahel', 'label': 'region'}",2009-01-14,"(261, 279)",precrise,Le Comité permanent inter-Etats de lutte contr...,Le Comité permanent inter-Etats de lutte contr...,...,['Insécurité alimentaire en milieu urbain : La...,False,2009-01-15,2009-01-14,Insécurité alimentaire en milieu urbain : La v...,2009-01-15,Publié le 2009-01-15 au Burkina : Insécurité a...,"{'text': 'le 14 janvier 2009', 'start': 261, '...",vulnérabilité - Sahel - le 14 janvier 2009,0
3,0,sociopolitique,insécurité,insécurité,"{'text': 'Ouagadougou', 'label': 'departement'}",2009-01-14,"(261, 279)",precrise,“La ville de Ouagadougou est menacée par l’ins...,“La ville de Ouagadougou est menacée par l’ins...,...,['Insécurité alimentaire en milieu urbain : La...,True,2009-01-15,2009-01-14,jeudi 15 janvier 2009 à 02h23min. Le Comité pe...,2009-01-15,Publié le 2009-01-15 au Burkina : jeudi 15 jan...,"{'text': 'le 14 janvier 2009', 'start': 261, '...",insécurité - Ouagadougou - le 14 janvier 2009,1
4,0,sociopolitique,vulnérabilité,vulnérabilité,"{'text': 'Ouagadougou', 'label': 'departement'}",2009-01-14,"(943, 961)",precrise,C’était lors de l’atelier de restitution des r...,C’était lors de l’atelier de restitution des r...,...,['Insécurité alimentaire en milieu urbain : La...,False,2009-01-15,2009-01-14,Car le nombre de repas a été réduit et la qual...,2009-01-15,Publié le 2009-01-15 au Burkina : Car le nombr...,"{'text': 'le 14 janvier 2009', 'start': 943, '...",vulnérabilité - Ouagadougou - le 14 janvier 2009,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,982,agriculture,organismes invasifs et pestes animales,chenille,"{'text': 'Bissa', 'label': 'village'}",2009-05-05,"(0, 0)",PRE_CRISE,Les Bobo ont mystifié les Peulh avec leurs che...,Les Bobo ont mystifié les Peulh avec leurs che...,...,['Journées de la parenté à plaisanterie : Les ...,False,2009-05-05,2009-05-05,Il est également prévu des assises nationales ...,2009-05-05,Publié le 2009-05-05 au Burkina : Il est égale...,"{'text': '2009-05-05', 'start': 0, 'end': 0}",chenille - Bissa - 2009-05-05,0
1153,984,sociopolitique,violence,guerre,"{'text': 'Burkina', 'label': 'country'}",2009-12-24,"(0, 0)",PRE_CRISE,Et le ministre des Affaires étrangères françai...,Et le ministre des Affaires étrangères françai...,...,['KOUCHNER ET LA GUINEE : Rhétorique trop hain...,False,2009-12-24,2009-12-24,jeudi 24 décembre 2009 à 01h07min. L’ambassade...,2009-12-24,Publié le 2009-12-24 au Burkina : jeudi 24 déc...,"{'text': '2009-12-24', 'start': 0, 'end': 0}",guerre - Burkina - 2009-12-24,0
1154,994,sociopolitique,instabilité politique,putsch,"{'text': 'Burkina', 'label': 'country'}",2009-11-09,"(0, 0)",PRE_CRISE,Toujours est-il que leur attitude 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ast
from matplotlib.patches import Rectangle

plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9

colors_palette = ['#2E4057', '#048A81', '#54C6EB', '#F18F01', '#C73E1D', '#8B5A3C', '#7209B7', '#A4031F']

def extract_location_info(location_str):
    """Robustly extract location text and label from a location string."""
    try:
        if pd.isna(location_str) or location_str == '':
            return None, None
        try:
            location_dict = ast.literal_eval(location_str)
            return location_dict.get('text', ''), location_dict.get('label', '')
        except:
            pass
        try:
            import json
            json_str = str(location_str).replace("'", '"')
            location_dict = json.loads(json_str)
            return location_dict.get('text', ''), location_dict.get('label', '')
        except:
            pass
        import re
        text_match = re.search(r"'text':\s*'([^']*)'", str(location_str))
        label_match = re.search(r"'label':\s*'([^']*)'", str(location_str))
        if text_match and label_match:
            return text_match.group(1), label_match.group(1)
        return None, None
    except Exception as e:
        print(f"Parsing error for: {str(location_str)[:50]}... -> {e}")
        return None, None

def create_figure_with_title(title, figsize=(12, 8)):
    """Create a figure with a main title."""
    fig = plt.figure(figsize=figsize)
    fig.suptitle(title, fontsize=14, fontweight='bold', y=0.98)
    return fig

# DATA DIAGNOSTIC
print("=== DATA STRUCTURE DIAGNOSTIC ===")
print(f"Total records: {len(df_preprocessed)}")
print(f"Available columns: {list(df_preprocessed.columns)}")

print("\n--- Sample of 'location' column ---")
sample_locations = df_preprocessed['location'].head(3)
for i, loc in enumerate(sample_locations):
    print(f"Example {i+1}: {loc}")
    print(f"Type: {type(loc)}")
    print(f"Length: {len(str(loc)) if loc is not None else 'N/A'}")
    print("---")

print("=== COMPLETE STATISTICAL ANALYSIS ===\n")

# Extract location information
print("Extracting location information...")
location_data = df_preprocessed['location'].apply(extract_location_info)
df_preprocessed['lieu_text'] = location_data.apply(lambda x: x[0] if x[0] else 'Unspecified')
df_preprocessed['lieu_label'] = location_data.apply(lambda x: x[1] if x[1] else 'Unspecified')

print(f"\nAfter extraction:")
print(f"Unique locations extracted: {df_preprocessed['lieu_text'].nunique()}")
print(f"Unique location types: {df_preprocessed['lieu_label'].nunique()}")
print(f"Percentage 'Unspecified' for locations: {(df_preprocessed['lieu_text']=='Unspecified').mean()*100:.1f}%")
print(f"Percentage 'Unspecified' for types: {(df_preprocessed['lieu_label']=='Unspecified').mean()*100:.1f}%")

if (df_preprocessed['lieu_text']=='Unspecified').mean() > 0.9:
    print("\nWARNING: More than 90% of data is 'Unspecified'")
    for i in range(min(3, len(df_preprocessed))):
        print(f"Row {i}: location = {df_preprocessed.iloc[i]['location']}")
        print(f"         -> Extraction result: {extract_location_info(df_preprocessed.iloc[i]['location'])}")
        print("---")

# ========== 1. LOCATION AND TYPE DISTRIBUTION ==========
print("\n1. Location distribution analysis...")
fig = create_figure_with_title("Distribution of Records by Locations and Location Types", (16, 10))

# Top 15 locations
plt.subplot(2, 2, 1)
lieu_counts = df_preprocessed['lieu_text'].value_counts().head(15)
bars1 = plt.bar(range(len(lieu_counts)), lieu_counts.values, color=colors_palette[0], alpha=0.7)
plt.title('Top 15 Most Frequent Locations', fontweight='bold')
plt.xlabel('Locations')
plt.ylabel('Number of records')
plt.xticks(range(len(lieu_counts)), lieu_counts.index, rotation=45, ha='right')
for i, bar in enumerate(bars1):
    plt.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.5, str(lieu_counts.values[i]), ha='center', va='bottom', fontsize=8)

# Distribution by location type
plt.subplot(2, 2, 2)
label_counts = df_preprocessed['lieu_label'].value_counts()
bars2 = plt.bar(range(len(label_counts)), label_counts.values, color=colors_palette[1], alpha=0.7)
plt.title('Distribution by Location Type', fontweight='bold')
plt.xlabel('Location types')
plt.ylabel('Number of records')
plt.xticks(range(len(label_counts)), label_counts.index, rotation=45, ha='right')
for i, bar in enumerate(bars2):
    plt.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.5, str(label_counts.values[i]), ha='center', va='bottom', fontsize=8)

# Percentage distribution of location types
plt.subplot(2, 2, 3)
perc_label = label_counts / label_counts.sum() * 100
bars3 = plt.bar(range(len(perc_label)), perc_label.values, color=colors_palette[1], alpha=0.7)
plt.title('Percentage Distribution of Location Types', fontweight='bold')
plt.xlabel('Location types')
plt.ylabel('Percentage (%)')
plt.xticks(range(len(perc_label)), perc_label.index, rotation=45, ha='right')
for i, bar in enumerate(bars3):
    plt.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.5, f"{perc_label.values[i]:.1f}%", ha='center', va='bottom', fontsize=8)

# Stacked bar: Top 10 locations vs location types
plt.subplot(2, 2, 4)
top_lieux = df_preprocessed['lieu_text'].value_counts().head(10).index
ct1 = pd.crosstab(df_preprocessed['lieu_text'], df_preprocessed['lieu_label']).loc[top_lieux]
bottom = np.zeros(len(ct1))
for i, lbl in enumerate(ct1.columns):
    plt.bar(range(len(ct1)), ct1[lbl], bottom=bottom, label=lbl, color=colors_palette[i % len(colors_palette)], alpha=0.8)
    bottom += ct1[lbl]
plt.title('Location Types for Top 10 Locations', fontweight='bold')
plt.xlabel('Locations')
plt.ylabel('Number of records')
plt.xticks(range(len(ct1)), ct1.index, rotation=45, ha='right')
plt.legend(bbox_to_anchor=(1.05,1), loc='upper left', fontsize=8)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# ========== 2. TERM ANALYSIS ==========
print("\n2. Term and association analysis...")
fig = create_figure_with_title("Term and Association Analysis", (16, 12))

# Top 15 terms
plt.subplot(2, 3, 1)
term_counts = df_preprocessed['term'].value_counts().head(15)
bars4 = plt.bar(range(len(term_counts)), term_counts.values, color=colors_palette[2], alpha=0.7)
plt.title('Top 15 Most Frequent Terms', fontweight='bold')
plt.xlabel('Terms')
plt.ylabel('Number of records')
plt.xticks(range(len(term_counts)), term_counts.index, rotation=45, ha='right')
for i, bar in enumerate(bars4):
    plt.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.5, str(term_counts.values[i]), ha='center', va='bottom', fontsize=8)

# Top 10 term + location associations
plt.subplot(2, 3, 2)
df_preprocessed['term_lieu'] = df_preprocessed['term'] + ' - ' + df_preprocessed['lieu_text']
term_lieu_counts = df_preprocessed['term_lieu'].value_counts().head(10)
bars5 = plt.barh(range(len(term_lieu_counts)), term_lieu_counts.values, color=colors_palette[3], alpha=0.7)
plt.title('Top 10 Term + Location Associations', fontweight='bold')
plt.xlabel('Number of records')
plt.ylabel('Associations')
plt.yticks(range(len(term_lieu_counts)), [x[:30] + '...' if len(x)>30 else x for x in term_lieu_counts.index])
for i, bar in enumerate(bars5):
    plt.text(bar.get_width()+0.5, bar.get_y()+bar.get_height()/2, str(term_lieu_counts.values[i]), ha='left', va='center', fontsize=8)

# Top 10 term + location type associations
plt.subplot(2, 3, 3)
df_preprocessed['term_label'] = df_preprocessed['term'] + ' - ' + df_preprocessed['lieu_label']
term_label_counts = df_preprocessed['term_label'].value_counts().head(10)
bars6 = plt.barh(range(len(term_label_counts)), term_label_counts.values, color=colors_palette[4], alpha=0.7)
plt.title('Top 10 Term + Location Type Associations', fontweight='bold')
plt.xlabel('Number of records')
plt.ylabel('Associations')
plt.yticks(range(len(term_label_counts)), term_label_counts.index)
for i, bar in enumerate(bars6):
    plt.text(bar.get_width()+0.5, bar.get_y()+bar.get_height()/2, str(term_label_counts.values[i]), ha='left', va='center', fontsize=8)

# Stacked bar: Top 8 terms vs location types
plt.subplot(2, 3, 4)
top_terms = df_preprocessed['term'].value_counts().head(8).index
ct2 = pd.crosstab(df_preprocessed['term'], df_preprocessed['lieu_label']).loc[top_terms]
bottom = np.zeros(len(ct2))
for i, lbl in enumerate(ct2.columns):
    plt.bar(range(len(ct2)), ct2[lbl], bottom=bottom, label=lbl, color=colors_palette[i % len(colors_palette)], alpha=0.8)
    bottom += ct2[lbl]
plt.title('Location Types for Top 8 Terms', fontweight='bold')
plt.xlabel('Terms')
plt.ylabel('Number of records')
plt.xticks(range(len(ct2)), ct2.index, rotation=45, ha='right')
plt.legend(bbox_to_anchor=(1.05,1), loc='upper left', fontsize=8)

# Top 8 terms pie chart
plt.subplot(2, 3, 5)
top_terms_pie = term_counts.head(8)
plt.pie(top_terms_pie.values, labels=top_terms_pie.index, autopct='%1.1f%%', colors=colors_palette[:len(top_terms_pie)], startangle=90)
plt.title('Distribution of Top 8 Terms', fontweight='bold')

# Terms distribution by location type
plt.subplot(2, 3, 6)
pivot_term_label = ct2.T
bottom = np.zeros(len(pivot_term_label))
for i, term in enumerate(pivot_term_label.columns):
    plt.bar(pivot_term_label.index, pivot_term_label[term], bottom=bottom, label=term, color=colors_palette[i % len(colors_palette)], alpha=0.8)
    bottom += pivot_term_label[term]
plt.title('Term Distribution by Location Type', fontweight='bold')
plt.xlabel('Location types')
plt.ylabel('Number of records')
plt.xticks(rotation=45, ha='right')
plt.legend(bbox_to_anchor=(1.05,1), loc='upper left', fontsize=8)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# ========== 3. LABEL DISTRIBUTIONS ==========
print("\n3. Label distribution analysis...")
fig = create_figure_with_title("Distributions by Label (Location Type)", (16, 10))

# Simple label distribution
plt.subplot(2, 2, 1)
label_dist = df_preprocessed['label'].value_counts()
bars7 = plt.bar(range(len(label_dist)), label_dist.values, color=colors_palette[5], alpha=0.7)
plt.title('Record Distribution by Label', fontweight='bold')
plt.xlabel('Labels')
plt.ylabel('Number of records')
plt.xticks(range(len(label_dist)), label_dist.index, rotation=45, ha='right')
for i, bar in enumerate(bars7):
    plt.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.5, str(label_dist.values[i]), ha='center', va='bottom', fontsize=8)

# Top 10 Label + Location combinations
plt.subplot(2, 2, 2)
df_preprocessed['label_lieu'] = df_preprocessed['label'].astype(str) + ' - ' + df_preprocessed['lieu_text']
label_lieu_counts = df_preprocessed['label_lieu'].value_counts().head(10)
bars8 = plt.barh(range(len(label_lieu_counts)), label_lieu_counts.values, color=colors_palette[6], alpha=0.7)
plt.title('Top 10 Label + Location Combinations', fontweight='bold')
plt.xlabel('Number of records')
plt.ylabel('Combinations')
plt.yticks(range(len(label_lieu_counts)), [x[:25] + '...' if len(x)>25 else x for x in label_lieu_counts.index])
for i, bar in enumerate(bars8):
    plt.text(bar.get_width()+0.5, bar.get_y()+bar.get_height()/2, str(label_lieu_counts.values[i]), ha='left', va='center', fontsize=8)

# Top 10 Label + Location Type combinations
plt.subplot(2, 2, 3)
df_preprocessed['label_type_lieu'] = df_preprocessed['label'].astype(str) + ' - ' + df_preprocessed['lieu_label']
label_type_counts = df_preprocessed['label_type_lieu'].value_counts().head(10)
bars9 = plt.barh(range(len(label_type_counts)), label_type_counts.values, color=colors_palette[7], alpha=0.7)
plt.title('Top 10 Label + Location Type Combinations', fontweight='bold')
plt.xlabel('Number of records')
plt.ylabel('Combinations')
plt.yticks(range(len(label_type_counts)), label_type_counts.index)
for i, bar in enumerate(bars9):
    plt.text(bar.get_width()+0.5, bar.get_y()+bar.get_height()/2, str(label_type_counts.values[i]), ha='left', va='center', fontsize=8)

# Top 10 Label + Term combinations
plt.subplot(2, 2, 4)
df_preprocessed['label_term'] = df_preprocessed['label'].astype(str) + ' - ' + df_preprocessed['term']
label_term_counts = df_preprocessed['label_term'].value_counts().head(10)
bars10 = plt.barh(range(len(label_term_counts)), label_term_counts.values, color=colors_palette[0], alpha=0.7)
plt.title('Top 10 Label + Term Combinations', fontweight='bold')
plt.xlabel('Number of records')
plt.ylabel('Combinations')
plt.yticks(range(len(label_term_counts)), [x[:30] + '...' if len(x)>30 else x for x in label_term_counts.index])
for i, bar in enumerate(bars10):
    plt.text(bar.get_width()+0.5, bar.get_y()+bar.get_height()/2, str(label_term_counts.values[i]), ha='left', va='center', fontsize=8)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# ========== 4. THEME DISTRIBUTION ==========
print("\n4. Theme distribution analysis...")
fig = create_figure_with_title("Record Distribution by Theme", (12, 8))

# Simple theme distribution
plt.subplot(2, 2, 1)
theme_counts = df_preprocessed['theme'].value_counts()
bars11 = plt.bar(range(len(theme_counts)), theme_counts.values, color=colors_palette[1], alpha=0.7)
plt.title('Distribution by Theme', fontweight='bold')
plt.xlabel('Themes')
plt.ylabel('Number of records')
plt.xticks(range(len(theme_counts)), theme_counts.index, rotation=45, ha='right')
for i, bar in enumerate(bars11):
    plt.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.5, str(theme_counts.values[i]), ha='center', va='bottom', fontsize=8)

# Percentage distribution by theme
plt.subplot(2, 2, 2)
perc_theme = theme_counts / theme_counts.sum() * 100
bars12 = plt.bar(range(len(perc_theme)), perc_theme.values, color=colors_palette[1], alpha=0.7)
plt.title('Percentage Distribution by Theme', fontweight='bold')
plt.xlabel('Themes')
plt.ylabel('Percentage (%)')
plt.xticks(range(len(perc_theme)), perc_theme.index, rotation=45, ha='right')
for i, bar in enumerate(bars12):
    plt.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.5, f"{perc_theme.values[i]:.1f}%", ha='center', va='bottom', fontsize=8)

# Stacked bar: Theme vs Top 6 terms
plt.subplot(2, 2, 3)
top_terms_theme = df_preprocessed['term'].value_counts().head(6).index
ct3 = pd.crosstab(df_preprocessed['theme'], df_preprocessed['term'])[top_terms_theme]
bottom = np.zeros(len(ct3))
for i, term in enumerate(ct3.columns):
    plt.bar(range(len(ct3)), ct3[term], bottom=bottom, label=term, color=colors_palette[i % len(colors_palette)], alpha=0.8)
    bottom += ct3[term]
plt.title('Top 6 Terms by Theme', fontweight='bold')
plt.xlabel('Themes')
plt.ylabel('Number of records')
plt.xticks(range(len(ct3)), ct3.index, rotation=45, ha='right')
plt.legend(bbox_to_anchor=(1.05,1), loc='upper left', fontsize=8)

# Stacked bar: Theme vs Location types
plt.subplot(2, 2, 4)
ct4 = pd.crosstab(df_preprocessed['theme'], df_preprocessed['lieu_label'])
bottom = np.zeros(len(ct4))
for i, lbl in enumerate(ct4.columns):
    plt.bar(range(len(ct4)), ct4[lbl], bottom=bottom, label=lbl, color=colors_palette[i % len(colors_palette)], alpha=0.8)
    bottom += ct4[lbl]
plt.title('Location Types by Theme', fontweight='bold')
plt.xlabel('Themes')
plt.ylabel('Number of records')
plt.xticks(range(len(ct4)), ct4.index, rotation=45, ha='right')
plt.legend(bbox_to_anchor=(1.05,1), loc='upper left', fontsize=8)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()



# ========== STATISTICAL SUMMARY ==========
print("\n" + "="*50)
print("COMPLETE STATISTICAL SUMMARY")
print("="*50)

print(f"\nGENERAL STATISTICS:")
print(f"   Total records: {len(df_preprocessed):,}")
print(f"   Unique locations: {df_preprocessed['lieu_text'].nunique()}")
print(f"   Unique location types: {df_preprocessed['lieu_label'].nunique()}")
print(f"   Unique terms: {df_preprocessed['term'].nunique()}")
print(f"   Unique themes: {df_preprocessed['theme'].nunique()}")
print(f"   Unique concepts: {df_preprocessed['concept'].nunique()}")

print("\nTOP ELEMENTS:")
print(f"   Most frequent location: {df_preprocessed['lieu_text'].value_counts().index[0]} ({df_preprocessed['lieu_text'].value_counts().iloc[0]} occurrences)")
print(f"   Dominant location type: {df_preprocessed['lieu_label'].value_counts().index[0]} ({df_preprocessed['lieu_label'].value_counts().iloc[0]} occurrences)")
print(f"   Most frequent term: {df_preprocessed['term'].value_counts().index[0]} ({df_preprocessed['term'].value_counts().iloc[0]} occurrences)")
print(f"   Dominant theme: {df_preprocessed['theme'].value_counts().index[0]} ({df_preprocessed['theme'].value_counts().iloc[0]} occurrences)")
print(f"   Dominant concept: {df_preprocessed['concept'].value_counts().index[0]} ({df_preprocessed['concept'].value_counts().iloc[0]} occurrences)")

print("\nMOST FREQUENT ASSOCIATIONS:")
print(f"   Term + Location: {term_lieu_counts.index[0]} ({term_lieu_counts.iloc[0]} occurrences)")
print(f"   Term + Type: {term_label_counts.index[0]} ({term_label_counts.iloc[0]} occurrences)")
print(f"   Label + Location: {label_lieu_counts.index[0]} ({label_lieu_counts.iloc[0]} occurrences)")

In [4]:
!python enrich_geo_wikidata_complete.py ./data/df_preprocessed.csv ./data/processed_wd.csv

ENRICHISSEMENT GÉOGRAPHIQUE VIA WIKIDATA
Burkina Faso - Analyse de sécurité alimentaire

📂 Fichier d'entrée: ./data/df_preprocessed.csv
📂 Fichier de sortie: ./data/processed_wd.csv

1. Chargement des données...
   ✓ 1157 enregistrements chargés

2. Enrichissement via Wikidata...
   (Cela peut prendre plusieurs minutes selon le nombre de lieux)
2026-02-06 00:35:51,715 - INFO - Traitement de 138 lieux uniques...
2026-02-06 00:35:51,715 - INFO - 
[1/138] Ouagadougou (departement)
2026-02-06 00:35:51,715 - INFO -   Recherche: Ouagadougou
2026-02-06 00:35:52,661 - INFO -     → Trouvé: Q3357901 (12.35, -1.516666666) - type: departement
2026-02-06 00:35:53,407 - INFO -     → Hiérarchie: None → Kadiogo → Kadiogo
2026-02-06 00:35:54,398 - INFO -     → 49 voisins dans 10km
2026-02-06 00:35:54,398 - INFO - 
[2/138] Sahel (region)
2026-02-06 00:35:54,398 - INFO -   Recherche: Sahel
2026-02-06 00:35:55,643 - INFO -     → Type Wikidata 'autre' ('zone géographique') → conservé label original 'region'

In [2]:
!python generate_neo4j_graph.py ./data/processed_wd.csv ./data/stkg_dtb.cypher


Input file: ./data/processed_wd.csv
Output file: ./data/stkg_dtb.cypher

Loading data...
  Total records: 1157

Preprocessing (normalizing locations and regions)...
  Location name corrections: 7 entries
    - 'Cascades' -> 'Tannounyan'
  Geographic type corrections: 19 entries
    - 'axe Sindou-Bobo-Dioulasso-Fara': 'nan' -> ''
    - 'Cascades': 'province' -> 'region'
    - 'Bamako': 'nan' -> ''
    - 'Burundi': 'nan' -> ''
    - 'Cuba': 'nan' -> ''
    - 'Guinée': 'nan' -> ''
    - 'Indochine': 'nan' -> ''
    - 'Europe': 'nan' -> ''
  Region name corrections: 583 entries
    - 'Kadiogo' -> 'Centre'
    - 'Goulmou' -> 'Est'
    - 'Liptako' -> 'Sahel'
    - 'Nando' -> 'Centre-Ouest'
    - 'Kuilsé' -> 'Centre-Nord'
    - 'Oubri' -> 'Plateau Central'
    - 'Yaadga' -> 'Nord'
    - 'Nakambé' -> 'Centre-Est'
    - 'Tannounyan' -> 'Cascades'
    - 'Bankui' -> 'Boucle du Mouhoun'
    - 'Nazinon' -> 'Centre-Sud'
    - 'Djôrô' -> 'Sud-Ouest'

Filtering data...
  After filtering (wikidata_id 